<h1> 2c. Refactoring to add batching and feature-creation </h1>

In this notebook, you continue reading the same small dataset, but refactor our ML pipeline in two small, but significant, ways:
<ol>
<li> Refactor the input to read data in batches.
<li> Refactor the feature creation so that it is not one-to-one with inputs.
</ol>

In [8]:
import google.datalab.ml as ml
import tensorflow as tf
from tensorflow.contrib import layers
print tf.__version__
#print ml.sdk_location

1.5.0


In [9]:
import datalab.bigquery as bq
import tensorflow as tf
import numpy as np
import shutil

<h2> 1. Refactor the input </h2>

Read data created in Lab1a, but this time make it more general, so that you are reading in batches.  Instead of using Pandas, you will add a filename queue to the TensorFlow graph.  This queue will be cycled through num_epochs times.

In [10]:
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
LABEL_COLUMN = 'fare_amount'
DEFAULTS = [[0.0], [-74.0], [40.0], [-74.0], [40.7], [1.0], ['nokey']]

def read_dataset(filename, num_epochs=None, batch_size=512, mode=tf.contrib.learn.ModeKeys.TRAIN):
  def _input_fn():
    input_file_names = tf.train.match_filenames_once(filename)
    filename_queue = tf.train.string_input_producer(
        input_file_names, num_epochs=num_epochs, shuffle=True)
    reader = tf.TextLineReader()
    _, value = reader.read_up_to(filename_queue, num_records=batch_size)

    value_column = tf.expand_dims(value, -1)
    columns = tf.decode_csv(value_column, record_defaults=DEFAULTS)
    features = dict(zip(CSV_COLUMNS, columns))
    label = features.pop(LABEL_COLUMN)
    return features, label

  return _input_fn

def get_train():
  return read_dataset('./taxi-train.csv', num_epochs=100, mode=tf.contrib.learn.ModeKeys.TRAIN)

def get_valid():
  return read_dataset('./taxi-valid.csv', num_epochs=1, mode=tf.contrib.learn.ModeKeys.EVAL)

def get_test():
  return read_dataset('./taxi-test.csv', num_epochs=1, mode=tf.contrib.learn.ModeKeys.EVAL)

<h2> 2. Refactor the way features are created. </h2>

For now, pass these through (same as previous lab).  However, refactoring this way will enable you to break the one-to-one relationship between inputs and features.

In [11]:
INPUT_COLUMNS = [
    layers.real_valued_column('pickuplon'),
    layers.real_valued_column('pickuplat'),
    layers.real_valued_column('dropofflat'),
    layers.real_valued_column('dropofflon'),
    layers.real_valued_column('passengers'),
]

feature_cols = INPUT_COLUMNS

<h2> Create and train the model </h2>

Note that the num_steps variable no longer exists.  get_train() specifies a num_epochs.

In [12]:
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree('taxi_trained', ignore_errors=True) # start fresh each time
model = tf.contrib.learn.LinearRegressor(
      feature_columns=feature_cols, model_dir='taxi_trained')
model.fit(input_fn=get_train());

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdf2864e490>, '_model_dir': 'taxi_trained', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 205.87857, step = 1
INFO:tensorflow:global_step/sec: 119.404
INFO:tensorflow:loss = 115.57962, step = 101 (0.842 sec)
INFO:tensorflow:global_step/sec: 119.302
INFO:tensorflow:loss = 71.51

<h3> Evaluate model </h3>

As before, evaluate on the validation data.  The third refactoring (to move the evaluation into the training loop) is in the next lab.

In [13]:
def print_rmse(model, name, input_fn):
  metrics = model.evaluate(input_fn=input_fn, steps=1)
  print 'RMSE on {} dataset = {}'.format(name, np.sqrt(metrics['loss']))
print_rmse(model, 'validation', get_valid())

INFO:tensorflow:Starting evaluation at 2018-03-01-15:34:00
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-1600
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2018-03-01-15:34:00
INFO:tensorflow:Saving dict for global step 1600: global_step = 1600, loss = 113.252846
RMSE on validation dataset = 10.6420316696


Copyright 2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License